# Load packages

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

# Set up paths

In [ ]:
code_dir = Path.cwd()
statistics_dir = code_dir.parent
source_dir = statistics_dir / "input"
output_dir = statistics_dir / "output/sensitivity_analysis/sex"
output_dir.mkdir(exist_ok=True, parents=True)

# Load data

In [ ]:
statistics_df = pd.read_csv(source_dir / 'statistics_df_randomized.csv', index_col = "sub_id")

# Set up R environment

In [ ]:
import os
import rpy2.robjects as robjects

# Set the R_HOME environment variable
os.environ['R_HOME'] = '/usr/lib/R/'

# Update the library paths
new_path = "/home/csi/R/x86_64-pc-linux-gnu-library/4.3"
robjects.r(f'.libPaths(c("{new_path}", .libPaths()))')


# Linear mixed effects models comparing BBB leakage between lesion, penumbra and normal tissue

In [ ]:
# Prepare dataframe
repeatead_measures = ['nice_normal_z_ef','nice_penumbra_z_ef','nice_lesion_z_ef']
df_lme = pd.melt(statistics_df.reset_index(), 
                 id_vars=['sub_id','AGE', 'SEX', 'NIHSSSCORE_V00', 'stroke_volume_v00', 'scanner'], 
                 value_vars=repeatead_measures, 
                 var_name="loc_measurement", 
                 value_name="EF", ignore_index=False).reset_index(drop=True)

# Separate df_lme according to sex
df_lme_m = df_lme[df_lme['SEX'] == 1]
df_lme_f = df_lme[df_lme['SEX'] == 2]

# Z-score age, stroke volume, NIHSS and EF
columns_to_normalize = ['AGE', 'stroke_volume_v00', 'NIHSSSCORE_V00', 'EF']
df_lme[columns_to_normalize] = df_lme[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())
df_lme_m[columns_to_normalize] = df_lme_m[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())
df_lme_f[columns_to_normalize] = df_lme_f[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())

## Male

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("EF ~ loc_measurement + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + (1|sub_id) + (1|scanner)", data=df_lme_m)

# Fit LMM 
lme = model.fit(factors={"loc_measurement": ['nice_normal_z_ef','nice_lesion_z_ef', 'nice_penumbra_z_ef']})
lme.to_csv(output_dir / "lesion_penumbra_normal_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "lesion_penumbra_normal_male_anova.csv")
anova

In [ ]:
# Compute post-hoc tests
marginal_estimates, comparisons = model.post_hoc(marginal_vars="loc_measurement", grouping_vars="loc_measurement")

# "Cell" means of the ANOVA
comparisons.to_csv(output_dir/"lesion_penumbra_normal_male_posthoc.csv")
print(comparisons)

## Visualization

In [ ]:
location_styled = [
    'Normal Tissue',
    'Penumbra (Tmax>6s)',
    'Infarct Core'
]

ef_styled = ['EF (z-scored)']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

repeatead_measures = ['nice_normal_z_ef','nice_penumbra_z_ef','nice_lesion_z_ef']
df_lme = pd.melt(statistics_df.reset_index(), 
                 id_vars=['sub_id','AGE', 'SEX', 'NIHSSSCORE_V00', 'stroke_volume_v00', 'scanner'], 
                 value_vars=repeatead_measures, 
                 var_name="loc_measurement", 
                 value_name="EF", ignore_index=False).reset_index(drop=True)

df_lme = df_lme[df_lme["SEX"] == 1]

custom_palette = sns.color_palette("Paired")


plt.figure(figsize=(8, 6))
sns.boxplot(y='EF', x='loc_measurement', data=df_lme, width=0.5, palette=custom_palette, boxprops=dict(alpha=0.7), showfliers=False)
sns.stripplot(y='EF', x='loc_measurement', data=df_lme, color='black', size=3, jitter=True, alpha=0.6)

# Annotations for significance
y_max = df_lme['EF'].max()
y_step = y_max * 0.12  # Calculate step size for annotations based on the maximum EF value

# Ensure the calculation of 'y' is correct by explicitly converting 'i' and 'y_step' to compatible types
for i, row in comparisons.iterrows():
    groups = row['Contrast'].split(' - ')
    p_value = row['P-val']
    
    # Assuming group names in 'loc_measurement' match those in 'Comparison'
    group_labels = df_lme['loc_measurement'].unique().tolist()
    x1 = group_labels.index(groups[0])
    x2 = group_labels.index(groups[1])
    
    # Correctly calculate 'y' by ensuring 'i' and 'y_step' are compatible types
    y = y_max + ((float(i)) * y_step)  # Added (i+1) to ensure spacing starts above the max value
    
    # Adjust 'p_text' based on your significance criteria
    p_text = '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'
    
    plt.plot([x1, x1, x2, x2], [y - y_step/4, y, y, y - y_step/4], lw=1.5, c='black')
    plt.text((x1 + x2) * 0.5, y, p_text, ha='center', va='bottom')


plt.xticks(ticks=np.arange(len(location_styled)), labels=location_styled, size=10)
plt.xlabel('')
plt.ylabel('EF (z-scored)')

plt.savefig(output_dir/"boxplot_lesion_penumbra_normal_male_ef.png", dpi=300)


## Female

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("EF ~ loc_measurement + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + (1|sub_id) + (1|scanner)", data=df_lme_f)

# Fit LMM 
lme = model.fit(factors={"loc_measurement": ['nice_normal_z_ef','nice_lesion_z_ef', 'nice_penumbra_z_ef']})
lme.to_csv(output_dir / "lesion_penumbra_normal_female_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "lesion_penumbra_normal_female_anova.csv")
anova

In [ ]:
# Compute post-hoc tests
marginal_estimates, comparisons = model.post_hoc(marginal_vars="loc_measurement", grouping_vars="loc_measurement")

# "Cell" means of the ANOVA
comparisons.to_csv(output_dir/"lesion_penumbra_normal_female_posthoc.csv")
print(comparisons)

## Visualization

In [ ]:
location_styled = [
    'Normal Tissue',
    'Penumbra (Tmax>6s)',
    'Infarct Core'
]

ef_styled = ['EF (z-scored)']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

repeatead_measures = ['nice_normal_z_ef','nice_penumbra_z_ef','nice_lesion_z_ef']
df_lme = pd.melt(statistics_df.reset_index(), 
                 id_vars=['sub_id','AGE', 'SEX', 'NIHSSSCORE_V00', 'stroke_volume_v00', 'scanner'], 
                 value_vars=repeatead_measures, 
                 var_name="loc_measurement", 
                 value_name="EF", ignore_index=False).reset_index(drop=True)

df_lme = df_lme[df_lme["SEX"] == 2]

custom_palette = sns.color_palette("Paired")


plt.figure(figsize=(8, 6))
sns.boxplot(y='EF', x='loc_measurement', data=df_lme, width=0.5, palette=custom_palette, boxprops=dict(alpha=0.7), showfliers=False)
sns.stripplot(y='EF', x='loc_measurement', data=df_lme, color='black', size=3, jitter=True, alpha=0.6)

# Annotations for significance
y_max = df_lme['EF'].max()
y_step = y_max * 0.12  # Calculate step size for annotations based on the maximum EF value

# Ensure the calculation of 'y' is correct by explicitly converting 'i' and 'y_step' to compatible types
for i, row in comparisons.iterrows():
    groups = row['Contrast'].split(' - ')
    p_value = row['P-val']
    
    # Assuming group names in 'loc_measurement' match those in 'Comparison'
    group_labels = df_lme['loc_measurement'].unique().tolist()
    x1 = group_labels.index(groups[0])
    x2 = group_labels.index(groups[1])
    
    # Correctly calculate 'y' by ensuring 'i' and 'y_step' are compatible types
    y = y_max + ((float(i)) * y_step)  # Added (i+1) to ensure spacing starts above the max value
    
    # Adjust 'p_text' based on your significance criteria
    p_text = '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'
    
    plt.plot([x1, x1, x2, x2], [y - y_step/4, y, y, y - y_step/4], lw=1.5, c='black')
    plt.text((x1 + x2) * 0.5, y, p_text, ha='center', va='bottom')


plt.xticks(ticks=np.arange(len(location_styled)), labels=location_styled, size=10)
plt.xlabel('')
plt.ylabel('EF (z-scored)')

plt.savefig(output_dir/"boxplot_lesion_penumbra_normal_female_ef.png", dpi=300)


# Linear mixed effects models comparing BBB leakage within the perfusion deficit

In [ ]:
# Prepare dataframe
statistics_df_clean = statistics_df.dropna(subset=['nice_penumbra_z_ef'])
repeatead_measures = ['nice_tmax6_z_ef','nice_tmax8_z_ef', 'nice_tmax10_z_ef']
df_lme = pd.melt(statistics_df_clean.reset_index(), 
                 id_vars=['sub_id','AGE', 'SEX', 'NIHSSSCORE_V00', 'stroke_volume_v00', 'scanner'], 
                 value_vars=repeatead_measures, 
                 var_name="loc_measurement", 
                 value_name="EF", ignore_index=False).reset_index(drop=True)

# Separate df_lme according to sex
df_lme_m = df_lme[df_lme['SEX'] == 1]
df_lme_f = df_lme[df_lme['SEX'] == 2]

# Z-score age, stroke volume, NIHSS and EF
columns_to_normalize = ['AGE', 'stroke_volume_v00', 'NIHSSSCORE_V00', 'EF']
df_lme[columns_to_normalize] = df_lme[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())
df_lme_m[columns_to_normalize] = df_lme_m[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())
df_lme_f[columns_to_normalize] = df_lme_f[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())

## Male

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("EF ~ loc_measurement + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + (1|sub_id) + (1|scanner)", data=df_lme_m)

# Fit LMM 
lme = model.fit(factors={"loc_measurement": ['nice_tmax6_z_ef','nice_tmax8_z_ef', 'nice_tmax10_z_ef']})
lme.to_csv(output_dir / "perfdef_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "perfdef_male_anova.csv")
anova

In [ ]:
# Compute post-hoc tests
marginal_estimates, comparisons = model.post_hoc(marginal_vars="loc_measurement", grouping_vars="loc_measurement")

# "Cell" means of the ANOVA
comparisons.to_csv(output_dir/"perfdef_male_posthoc.csv")
print(comparisons)

In [ ]:
location_styled = [
    'Tmax 6-8s',
    'Tmax 8-10s',
    'Tmax >/=10s'
]

ef_styled = ['EF (z-scored)']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

statistics_df_clean = statistics_df.dropna(subset=['nice_penumbra_z_ef'])
repeatead_measures = ['nice_tmax6_z_ef','nice_tmax8_z_ef', 'nice_tmax10_z_ef']
df_lme = pd.melt(statistics_df_clean.reset_index(), 
                 id_vars=['sub_id','AGE', 'SEX', 'NIHSSSCORE_V00', 'stroke_volume_v00', 'scanner'], 
                 value_vars=repeatead_measures, 
                 var_name="loc_measurement", 
                 value_name="EF", ignore_index=False).reset_index(drop=True)

custom_palette = sns.color_palette("Paired")

df_lme = df_lme[df_lme["SEX"] == 1]

plt.figure(figsize=(8, 6))
sns.boxplot(y='EF', x='loc_measurement', data=df_lme, width=0.5, palette=custom_palette, boxprops=dict(alpha=0.7), showfliers=False)
sns.stripplot(y='EF', x='loc_measurement', data=df_lme, color='black', size=3, jitter=True, alpha=0.6)

# Annotations for significance
y_max = df_lme['EF'].max()
y_step = y_max * 0.12  # Calculate step size for annotations based on the maximum EF value

# Ensure the calculation of 'y' is correct by explicitly converting 'i' and 'y_step' to compatible types
for i, row in comparisons.iterrows():
    groups = row['Contrast'].split(' - ')
    p_value = row['P-val']
    
    # Assuming group names in 'loc_measurement' match those in 'Comparison'
    group_labels = df_lme['loc_measurement'].unique().tolist()
    x1 = group_labels.index(groups[0])
    x2 = group_labels.index(groups[1])
    
    # Correctly calculate 'y' by ensuring 'i' and 'y_step' are compatible types
    y = y_max + ((float(i)) * y_step)  # Added (i+1) to ensure spacing starts above the max value
    
    # Adjust 'p_text' based on your significance criteria
    p_text = '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'
    
    plt.plot([x1, x1, x2, x2], [y - y_step/4, y, y, y - y_step/4], lw=1.5, c='black')
    plt.text((x1 + x2) * 0.5, y, p_text, ha='center', va='bottom')


plt.xticks(ticks=np.arange(len(location_styled)), labels=location_styled, size=10)
plt.xlabel('')
plt.ylabel('EF (z-scored)')

plt.savefig(output_dir/"boxplot_perfdef_male_ef.png", dpi=300)


## Female

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("EF ~ loc_measurement + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + (1|sub_id) + (1|scanner)", data=df_lme_f)

# Fit LMM 
lme = model.fit(factors={"loc_measurement": ['nice_tmax6_z_ef','nice_tmax8_z_ef', 'nice_tmax10_z_ef']})
lme.to_csv(output_dir / "perfdef_female_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "perfdef_female_anova.csv")
anova

In [ ]:
# Compute post-hoc tests
marginal_estimates, comparisons = model.post_hoc(marginal_vars="loc_measurement", grouping_vars="loc_measurement")

# "Cell" means of the ANOVA
comparisons.to_csv(output_dir/"perfdef_female_posthoc.csv")
print(comparisons)

In [ ]:
location_styled = [
    'Tmax 6-8s',
    'Tmax 8-10s',
    'Tmax >/=10s'
]

ef_styled = ['EF (z-scored)']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

statistics_df_clean = statistics_df.dropna(subset=['nice_penumbra_z_ef'])
repeatead_measures = ['nice_tmax6_z_ef','nice_tmax8_z_ef', 'nice_tmax10_z_ef']
df_lme = pd.melt(statistics_df_clean.reset_index(), 
                 id_vars=['sub_id','AGE', 'SEX', 'NIHSSSCORE_V00', 'stroke_volume_v00', 'scanner'], 
                 value_vars=repeatead_measures, 
                 var_name="loc_measurement", 
                 value_name="EF", ignore_index=False).reset_index(drop=True)

custom_palette = sns.color_palette("Paired")

df_lme = df_lme[df_lme["SEX"] == 2]

plt.figure(figsize=(8, 6))
sns.boxplot(y='EF', x='loc_measurement', data=df_lme, width=0.5, palette=custom_palette, boxprops=dict(alpha=0.7), showfliers=False)
sns.stripplot(y='EF', x='loc_measurement', data=df_lme, color='black', size=3, jitter=True, alpha=0.6)

# Annotations for significance
y_max = df_lme['EF'].max()
y_step = y_max * 0.12  # Calculate step size for annotations based on the maximum EF value

# Ensure the calculation of 'y' is correct by explicitly converting 'i' and 'y_step' to compatible types
for i, row in comparisons.iterrows():
    groups = row['Contrast'].split(' - ')
    p_value = row['P-val']
    
    # Assuming group names in 'loc_measurement' match those in 'Comparison'
    group_labels = df_lme['loc_measurement'].unique().tolist()
    x1 = group_labels.index(groups[0])
    x2 = group_labels.index(groups[1])
    
    # Correctly calculate 'y' by ensuring 'i' and 'y_step' are compatible types
    y = y_max + ((float(i)) * y_step)  # Added (i+1) to ensure spacing starts above the max value
    
    # Adjust 'p_text' based on your significance criteria
    p_text = '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'
    
    plt.plot([x1, x1, x2, x2], [y - y_step/4, y, y, y - y_step/4], lw=1.5, c='black')
    plt.text((x1 + x2) * 0.5, y, p_text, ha='center', va='bottom')


plt.xticks(ticks=np.arange(len(location_styled)), labels=location_styled, size=10)
plt.xlabel('')
plt.ylabel('EF (z-scored)')

plt.savefig(output_dir/"boxplot_perfdef_female_ef.png", dpi=300)


# Linear mixed effects models comparing BBB leakage  within penumbra according to future infarction

In [ ]:
# EF dataframe
statistics_df_clean = statistics_df.dropna(subset=['nice_penumbra_z_ef'])
repeatead_measures = ['nice_penumbra_noinfarct_z_ef', 'nice_penumbra_infarct_z_ef']
df_ef = pd.melt(statistics_df_clean.reset_index(), 
                 id_vars=['sub_id','AGE', 'SEX', 'NIHSSSCORE_V00', 'stroke_volume_v00', 'treatment', 'scanner'], 
                 value_vars=repeatead_measures, 
                 var_name="loc_measurement", 
                 value_name="EF", ignore_index=False).reset_index(drop=True)
df_ef['loc_measurement'] = df_ef['loc_measurement'].apply(lambda x: x.split('_')[2])

# Tmax dataframe
tmax = ['nice_penumbra_noinfarct_mean_tmax_rapid', 'nice_penumbra_infarct_mean_tmax_rapid']
df_tmax = pd.melt(statistics_df_clean.reset_index(), 
                 id_vars='sub_id',
                    value_vars=tmax,
                    var_name='loc_measurement',
                    value_name='Tmax', ignore_index=False).reset_index(drop=True)

# Rename values of loc_measurement by splitting the string
df_tmax['loc_measurement'] = df_tmax['loc_measurement'].apply(lambda x: x.split('_')[2])

# Merge EF and Tmax dataframes
df_lme = pd.merge(df_ef, df_tmax, on=['sub_id', 'loc_measurement'])

# Separate df_lme according to sex
df_lme_m = df_lme[df_lme['SEX'] == 1]
df_lme_f = df_lme[df_lme['SEX'] == 2]

# Drop rows with NaN values in EF and Tmax
df_lme_m = df_lme_m.dropna(subset=['EF', 'Tmax'])
df_lme_f = df_lme_f.dropna(subset=['EF', 'Tmax'])
df_lme = df_lme.dropna(subset=['EF', 'Tmax'])

# Z-score age, stroke volume, NIHSS and EF
columns_to_normalize = ['AGE', 'stroke_volume_v00', 'NIHSSSCORE_V00', 'EF']
df_lme[columns_to_normalize] = df_lme[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())
df_lme_m[columns_to_normalize] = df_lme_m[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())
df_lme_f[columns_to_normalize] = df_lme_f[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())

## Male

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("EF ~ loc_measurement + Tmax + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:loc_measurement + (1|sub_id) + (1|scanner)", data=df_lme_m)

# Fit LMM 
lme = model.fit(factors={"loc_measurement": ['noinfarct', 'infarct']})
lme.to_csv(output_dir / "penumbra_adjusted_interaction_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_adjusted_interaction_male_anova.csv")
anova

In [ ]:
# Compute post-hoc tests
marginal_estimates, comparisons = model.post_hoc(marginal_vars="loc_measurement", grouping_vars="loc_measurement")

# "Cell" means of the ANOVA
comparisons.to_csv(output_dir/"penumbra_adjusted_interaction_male_posthoc.csv")
print(comparisons)

### PLACEBO

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("EF ~ loc_measurement + Tmax + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + (1|sub_id) + (1|scanner)", data=df_lme_m[df_lme_m['treatment'] == 0])

# Fit LMM 
lme = model.fit(factors={"loc_measurement": ['noinfarct', 'infarct']})
lme.to_csv(output_dir / "penumbra_adjusted_placebo_interaction_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_adjusted_placebo_interaction_male_anova.csv")
anova

In [ ]:
# Compute post-hoc tests
marginal_estimates, comparisons = model.post_hoc(marginal_vars="loc_measurement", grouping_vars="loc_measurement")

# "Cell" means of the ANOVA
comparisons.to_csv(output_dir/"penumbra_adjusted_placebo_interaction_male_posthoc.csv")
print(comparisons)

### TREATMENT

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("EF ~ loc_measurement + Tmax + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + (1|sub_id) + (1|scanner)", data=df_lme_m[df_lme_m['treatment'] == 1])

# Fit LMM 
lme = model.fit(factors={"loc_measurement": ['noinfarct', 'infarct']})
lme.to_csv(output_dir / "penumbra_adjusted_treatment_interaction_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_adjusted_treatment_interaction_male_anova.csv")
anova

In [ ]:
# Compute post-hoc tests
marginal_estimates, comparisons = model.post_hoc(marginal_vars="loc_measurement", grouping_vars="loc_measurement")

# "Cell" means of the ANOVA
comparisons.to_csv(output_dir/"penumbra_adjusted_treatment_interaction_male_posthoc.csv")
print(comparisons)


## Female

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("EF ~ loc_measurement + Tmax + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:loc_measurement + (1|sub_id) + (1|scanner)", data=df_lme_f)

# Fit LMM 
lme = model.fit(factors={"loc_measurement": ['noinfarct', 'infarct']})
lme.to_csv(output_dir / "penumbra_adjusted_interaction_female_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_adjusted_interaction_female_anova.csv")
anova

In [ ]:
# Compute post-hoc tests
marginal_estimates, comparisons = model.post_hoc(marginal_vars="loc_measurement", grouping_vars="loc_measurement")

# "Cell" means of the ANOVA
comparisons.to_csv(output_dir/"penumbra_adjusted_interaction_female_posthoc.csv")
print(comparisons)

# Future-infarction Penumbra: Voxel-wise analysis

In [ ]:
# load dataframe
fip_df = pd.read_csv(source_dir / 'statistics_df_futureinfarction.csv')

# Separate df_lme according to sex
fip_df_m = fip_df[fip_df['SEX'] == 1]
fip_df_f = fip_df[fip_df['SEX'] == 2]

# Z-score age, stroke volume, NIHSS and EF
columns_to_normalize = ['AGE', 'stroke_volume_v00', 'NIHSSSCORE_V00', 'ef_value', 'tmax_value']
fip_df_f[columns_to_normalize] = fip_df_f[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())
fip_df_m[columns_to_normalize] = fip_df_m[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())
fip_df[columns_to_normalize] = fip_df[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())

## Infarct ~ EF

### Male

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("infarct ~ ef_value + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:ef_value + (ef_value|sub_id) + (ef_value|scanner)", data=fip_df_m, family="binomial")

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "penumbra_voxelwise_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_male_anova.csv")
anova

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("infarct ~ ef_value + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:ef_value + (ef_value|sub_id) + (ef_value|scanner)", data=fip_df_m, family="binomial")

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "penumbra_voxelwise_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_male_anova.csv")
anova

#### Placebo

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("infarct ~ ef_value + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:ef_value + (ef_value|sub_id) + (ef_value|scanner)", data=fip_df_m[fip_df_m["treatment"] == 0], family="binomial")

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "penumbra_voxelwise_placebo_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_placebo_male_anova.csv")
anova

#### Treatment

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("infarct ~ ef_value + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:ef_value + (ef_value|sub_id) + (ef_value|scanner)", data=fip_df_m[fip_df_m["treatment"] == 1], family="binomial")

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "penumbra_voxelwise_treatment_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_treatment_male_anova.csv")

### Female

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("infarct ~ ef_value + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:ef_value + (ef_value|sub_id) + (ef_value|scanner)", data=fip_df_f, family="binomial")

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "penumbra_voxelwise_female_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_female_anova.csv")
anova

## Infarct ~ EF + Tmax 

### Male

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("infarct ~ ef_value + tmax_value + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:ef_value + treatment:tmax_value + (ef_value + tmax_value|sub_id) + (ef_value + tmax_value|scanner)", data=fip_df_m, family="binomial")

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "penumbra_voxelwise_ef_tmax_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_ef_tmax_male_anova.csv")
anova

#### Placebo

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("infarct ~ ef_value + tmax_value + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:ef_value + treatment:tmax_value + (ef_value + tmax_value|sub_id) + (ef_value + tmax_value|scanner)", data=fip_df_m[fip_df_m["treatment"] == 0], family="binomial")

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "penumbra_voxelwise_ef_tmax_placebo_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_ef_tmax_placebo_male_anova.csv")
anova

#### Treatment

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("infarct ~ ef_value + tmax_value + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:ef_value + treatment:tmax_value + (ef_value + tmax_value|sub_id) + (ef_value + tmax_value|scanner)", data=fip_df_m[fip_df_m["treatment"] == 1], family="binomial")

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "penumbra_voxelwise_ef_tmax_treatment_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_ef_tmax_treatment_male_anova.csv")

### Female

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("infarct ~ ef_value + tmax_value + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:ef_value + treatment:tmax_value + (ef_value + tmax_value|sub_id) + (ef_value + tmax_value|scanner)", data=fip_df_f, family="binomial")

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "penumbra_voxelwise_ef_tmax_female_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_ef_tmax_female_anova.csv")
anova

## Infarct ~ Tmax

### Male

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("infarct ~ tmax_value + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:tmax_value + (tmax_value|sub_id) + (tmax_value|scanner)", data=fip_df_m, family="binomial")

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "penumbra_voxelwise_tmax_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_tmax_male_anova.csv")
anova

### Female

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("infarct ~ tmax_value + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:tmax_value + (tmax_value|sub_id) + (tmax_value|scanner)", data=fip_df_f, family="binomial")

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "penumbra_voxelwise_tmax_female_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_tmax_female_anova.csv")
anova

## EF ~ Tmax

### Male

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("ef_value ~ tmax_value + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + (tmax_value|sub_id) + (tmax_value|scanner)", data=fip_df_m)

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "penumbra_voxelwise_tmax_ef_corr_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_tmax_ef_corr_male_anova.csv")
anova

### Female

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("ef_value ~ tmax_value + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + (tmax_value|sub_id) + (tmax_value|scanner)", data=fip_df_f)

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "penumbra_voxelwise_tmax_ef_corr_female_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_tmax_ef_corr_female_anova.csv")
anova

# Models with clinical outcomes

## Set up dataframe

In [ ]:
from scipy.stats import zscore

clinical_variables = [
    'AGE', 
    'SEX', 
    'NIHSSSCORE_V00', 
    'stroke_volume_v00',
    'treatment', 
    'scanner',
    'SABCRIT2', 
    'excellent_outcome',
    'NIHSSSCORE_V05'
]

imaging_variables = [
    'nice_normal_z_ef', 
    'nice_penumbra_z_ef',
    'nice_lesion_z_ef'
]

continuous_variables = imaging_variables + ['AGE', 'NIHSSSCORE_V00', 'NIHSSSCORE_V05', 'stroke_volume_v00']

# Create df with clinical and imaging variables
df_clinical = statistics_df[clinical_variables + imaging_variables]
df_clinical_penumbra = df_clinical.dropna(subset=['nice_penumbra_z_ef'])

# Create separate dataframes for female and males
df_clinical_m = df_clinical[df_clinical["SEX"] == 1]
df_clinical_f = df_clinical[df_clinical["SEX"] == 2]
df_clinical_penumbra_m = df_clinical_penumbra[df_clinical_penumbra["SEX"] == 1]
df_clinical_penumbra_f = df_clinical_penumbra[df_clinical_penumbra["SEX"] == 2]

# Z-score dataframes
for col in continuous_variables:
    df_clinical[col] = zscore(df_clinical[col], nan_policy="omit")
    df_clinical_penumbra[col] = zscore(df_clinical_penumbra[col], nan_policy="omit")
    df_clinical_f[col] = zscore(df_clinical_f[col], nan_policy="omit")
    df_clinical_m[col] = zscore(df_clinical_m[col], nan_policy="omit")
    df_clinical_penumbra_f[col] = zscore(df_clinical_penumbra_f[col], nan_policy="omit")
    df_clinical_penumbra_m[col] = zscore(df_clinical_penumbra_m[col], nan_policy="omit")

## Logistic regression model with random effects: EF (lesion) ~ hemorrhage

### Male

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("SABCRIT2 ~ nice_lesion_z_ef + AGE + stroke_volume_v00 + NIHSSSCORE_V00 + treatment + treatment:nice_lesion_z_ef + (nice_lesion_z_ef|scanner)", data=df_clinical_m, family='binomial')

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "hemorrhage_ef_lesion_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "hemorrhage_ef_lesion_male_anova.csv")
anova

### Female

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("SABCRIT2 ~ nice_lesion_z_ef + AGE + stroke_volume_v00 + NIHSSSCORE_V00 + treatment + treatment:nice_lesion_z_ef + (nice_lesion_z_ef|scanner)", data=df_clinical_f, family='binomial')

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "hemorrhage_ef_lesion_female_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "hemorrhage_ef_lesion_female_anova.csv")
anova

## Logistic regression model with random effects: EF (penumbra) ~ hemorrhage

### Male

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("SABCRIT2 ~ nice_penumbra_z_ef + AGE + stroke_volume_v00 + NIHSSSCORE_V00 + treatment + treatment:nice_penumbra_z_ef + (nice_penumbra_z_ef|scanner)", data=df_clinical_penumbra_m, family='binomial')

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "hemorrhage_ef_penumbra_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "hemorrhage_ef_penumbra_male_anova.csv")
anova

### Female

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("SABCRIT2 ~ nice_penumbra_z_ef + AGE + stroke_volume_v00 + NIHSSSCORE_V00 + treatment + treatment:nice_penumbra_z_ef + (nice_penumbra_z_ef|scanner)", data=df_clinical_penumbra_f, family='binomial')

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "hemorrhage_ef_penumbra_female_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "hemorrhage_ef_penumbra_female_anova.csv")
anova

## Linear mixed effects model: EF (penumbra) ~ Excellent outcome (mRS 90 days 0-1)

### Male

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("excellent_outcome ~ nice_penumbra_z_ef + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:nice_penumbra_z_ef + (nice_penumbra_z_ef|scanner)", data=df_clinical_penumbra_m, family='binomial')

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "excellent_ef_penumbra_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "excellent_ef_penumbra_male_anova.csv")
anova

### Female

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("excellent_outcome ~ nice_penumbra_z_ef + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:nice_penumbra_z_ef + (nice_penumbra_z_ef|scanner)", data=df_clinical_penumbra_f, family='binomial')

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "excellent_ef_penumbra_female_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "excellent_ef_penumbra_female_anova.csv")
anova

## Linear mixed effects model: EF (lesion) ~ Excellent outcome (mRS 90 days 0-1)

### Male

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("excellent_outcome ~ nice_lesion_z_ef + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:nice_lesion_z_ef + (nice_lesion_z_ef|scanner)", data=df_clinical_m, family='binomial')

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "excellent_ef_lesion_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "excellent_ef_lesion_male_anova.csv")
anova

### Female

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("excellent_outcome ~ nice_lesion_z_ef + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:nice_lesion_z_ef + (nice_lesion_z_ef|scanner)", data=df_clinical_f, family='binomial')

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "excellent_ef_lesion_female_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "excellent_ef_lesion_female_anova.csv")
anova

## Linear mixed effects model: EF (penumbra) ~ NIHSS at 90 days

### Male

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("NIHSSSCORE_V05 ~ nice_penumbra_z_ef + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:nice_penumbra_z_ef + (nice_penumbra_z_ef|scanner)", data=df_clinical_penumbra_m)

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "nihss_ef_penumbra_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "nihss_ef_penumbra_male_anova.csv")
anova

### Female

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("NIHSSSCORE_V05 ~ nice_penumbra_z_ef + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:nice_penumbra_z_ef + (1 |scanner)", data=df_clinical_penumbra_f)

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "nihss_ef_penumbra_female_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "nihss_ef_penumbra_female_anova.csv")
anova


## Linear mixed effects model: EF (lesion) ~ NIHSS at 90 days

### Male

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("NIHSSSCORE_V05 ~ nice_lesion_z_ef + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:nice_lesion_z_ef + (nice_lesion_z_ef|scanner)", data=df_clinical_m)

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "nihss_ef_lesion_male_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "nihss_ef_lesion_male_anova.csv")
anova

### Female

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("NIHSSSCORE_V05 ~ nice_lesion_z_ef + AGE + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:nice_lesion_z_ef + (nice_lesion_z_ef|scanner)", data=df_clinical_f)

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "nihss_ef_lesion_female_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "nihss_ef_lesion_female_anova.csv")
anova